<a href="https://www.kaggle.com/code/kumarsakshat22/flood-prediction?scriptVersionId=179075843" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e5/sample_submission.csv
/kaggle/input/playground-series-s4e5/train.csv
/kaggle/input/playground-series-s4e5/test.csv
/kaggle/input/flood-prediction-factors/flood.csv


In [2]:
import pandas as pd 
import numpy as np 


In [3]:
df = pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv")
df_test = pd.read_csv("/kaggle/input/playground-series-s4e5/test.csv")
df_og = pd.read_csv("/kaggle/input/flood-prediction-factors/flood.csv")

In [4]:
len(df_og)

50000

In [5]:
len(df)

1117957

In [6]:
df.duplicated().sum()

0

In [7]:
df.isnull().sum()

id                                 0
MonsoonIntensity                   0
TopographyDrainage                 0
RiverManagement                    0
Deforestation                      0
Urbanization                       0
ClimateChange                      0
DamsQuality                        0
Siltation                          0
AgriculturalPractices              0
Encroachments                      0
IneffectiveDisasterPreparedness    0
DrainageSystems                    0
CoastalVulnerability               0
Landslides                         0
Watersheds                         0
DeterioratingInfrastructure        0
PopulationScore                    0
WetlandLoss                        0
InadequatePlanning                 0
PoliticalFactors                   0
FloodProbability                   0
dtype: int64

In [8]:
df_og.isnull().sum()

MonsoonIntensity                   0
TopographyDrainage                 0
RiverManagement                    0
Deforestation                      0
Urbanization                       0
ClimateChange                      0
DamsQuality                        0
Siltation                          0
AgriculturalPractices              0
Encroachments                      0
IneffectiveDisasterPreparedness    0
DrainageSystems                    0
CoastalVulnerability               0
Landslides                         0
Watersheds                         0
DeterioratingInfrastructure        0
PopulationScore                    0
WetlandLoss                        0
InadequatePlanning                 0
PoliticalFactors                   0
FloodProbability                   0
dtype: int64

In [9]:
df = pd.concat([df,df_og])

In [10]:
len(df)

1167957

In [11]:
df.drop("id", inplace = True, axis = 1)

In [12]:
len(df)

1167957

In [13]:
df.head()

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
0,5,8,5,8,6,4,4,3,3,4,...,5,3,3,5,4,7,5,7,3,0.445
1,6,7,4,4,8,8,3,5,4,6,...,7,2,0,3,5,3,3,4,3,0.450
2,6,5,6,7,3,7,1,5,4,5,...,7,3,7,5,6,8,2,3,3,0.530
3,3,4,6,5,4,8,4,7,6,8,...,2,4,7,4,4,6,5,7,5,0.535
4,5,3,2,6,4,4,3,3,3,3,...,2,2,6,6,4,1,2,3,5,0.415


In [14]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error

In [15]:
X = df.drop('FloodProbability', axis=1)
y = df['FloodProbability']

In [16]:
pip install optuna xgboost


Note: you may need to restart the kernel to use updated packages.


In [17]:
import optuna
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X[:3000], y[:3000], test_size=0.2, random_state=42)

# Define the objective function
def objective(trial):
    param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'booster': 'gbtree',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.0, 0.5),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True)
    }

    # Create the XGBoost model
    model = xgb.XGBRegressor(**param)

    # Evaluate the model using cross-validation
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    rmse = (-score.mean())**0.5
    return rmse

# Create study and optimize
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, n_jobs=-1)

# Print the best parameters
print("Best parameters found: ", study.best_params)

# Train the model with the best parameters on the entire training set
best_params = study.best_params
best_model = xgb.XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred = best_model.predict(X_test)
rmse = ((y_test - y_pred) ** 2).mean() ** 0.5
print(f"RMSE on test set: {rmse}")


[I 2024-05-22 07:15:04,541] A new study created in memory with name: no-name-6e023b92-e5c3-4408-93db-dcf2d1bc320e
[I 2024-05-22 07:15:07,822] Trial 3 finished with value: 0.04951260986687568 and parameters: {'n_estimators': 343, 'max_depth': 5, 'min_child_weight': 9, 'gamma': 0.21679695502509994, 'learning_rate': 0.26110759788610155, 'subsample': 0.8347581304827651, 'colsample_bytree': 0.5865705079544364, 'lambda': 5.1943439512105125e-06, 'alpha': 2.373459786843492e-05}. Best is trial 3 with value: 0.04951260986687568.
[I 2024-05-22 07:15:09,836] Trial 4 finished with value: 0.04699721338402372 and parameters: {'n_estimators': 208, 'max_depth': 6, 'min_child_weight': 3, 'gamma': 0.1358929817243022, 'learning_rate': 0.29658909640102465, 'subsample': 0.9664040128194025, 'colsample_bytree': 0.6449365428311904, 'lambda': 5.099479274430188e-05, 'alpha': 0.0003454997230510155}. Best is trial 4 with value: 0.04699721338402372.
[I 2024-05-22 07:15:10,834] Trial 0 finished with value: 0.0494860

Best parameters found:  {'n_estimators': 188, 'max_depth': 3, 'min_child_weight': 5, 'gamma': 0.0004979034056008056, 'learning_rate': 0.11210670083087085, 'subsample': 0.7347948037907287, 'colsample_bytree': 0.8508108772812317, 'lambda': 7.631611110098673e-05, 'alpha': 0.005260530314556171}
RMSE on test set: 0.026529245668254587


In [18]:
best_model_entire = xgb.XGBRegressor(**best_params)

In [19]:
best_model_entire.fit(X,y)

XGBRegressor(alpha=0.005260530314556171, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8508108772812317, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0.0004979034056008056,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=7.631611110098673e-05,
             learning_rate=0.11210670083087085, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=188, n_jobs=None, ...)

In [20]:
df_test.drop('id', axis =1, inplace =True)

In [21]:
y_pred = best_model_entire.predict(df_test)

In [22]:
sub = pd.read_csv("/kaggle/input/playground-series-s4e5/sample_submission.csv")

In [23]:
sub.head()

,id,FloodProbability
0,1117957,0.5
1,1117958,0.5
2,1117959,0.5
3,1117960,0.5
4,1117961,0.5


In [24]:
sub['FloodProbability'] = y_pred

In [25]:
sub.head()

,id,FloodProbability
0,1117957,0.559171
1,1117958,0.464144
2,1117959,0.466742
3,1117960,0.478467
4,1117961,0.472917


In [26]:
sub.to_csv('submission.csv', index=False)